In [ ]:
import os
import random
from variantes import (
    filas, columnas, agua, Fallo, tocado, barco, letras_columnas, flota,
    ARCHIVO_PARTIDA, ANSI_ROJO, ANSI_RESET
)
from funciones import (
    cargar_partida, guardar_partida, turno_jugador, turno_maquina,
    todos_barcos_hundidos, crear_tablero, colocar_barcos_manual,
    colocar_barcos_aleatorio_maquina, imprimir_tableros_lado
)

""" 
Cargar partida guardada o iniciar nueva.
"""
def juego_principal():

    estado = cargar_partida()
    
    if estado:
        print("Se ha encontrado una partida guardada.")
        opcion = input("¿Quieres reanudarla? (S/N): ").strip().upper()
        if opcion == 'S':
            (
                alias_jugador, tablero_defensa_jugador, barcos_jugador,
                tablero_ataque_jugador, tablero_defensa_maquina,
                barcos_maquina, tablero_ataque_maquina, turno_jugador_actual,
                contador_turno
            ) = estado
        else:
            estado = None

""" 
Configuración inicial de una nueva partida."""
    if not estado:
        print("=== El Hombre contra la Máquina ===")
        alias_jugador = input("Jugador 1: introduce tu Alias: ").strip() or "Jugador 1"
        
        barcos_jugador, tablero_defensa_jugador, _ = colocar_barcos_manual(alias_jugador)
        tablero_ataque_jugador = crear_tablero()
        
        barcos_maquina, tablero_defensa_maquina = colocar_barcos_aleatorio_maquina()
        tablero_ataque_maquina = crear_tablero()
        
        print("La máquina ha colocado sus barcos (posición secreta).")
        input("Pulsa Enter para continuar...")
        
""" 
Decidir quién comienza.
"""
        while True:
            eleccion = input(f"{alias_jugador}, elige 'Cara' o 'Cruz' (C/X): ").strip().upper()
            if eleccion in ("CARA", "C"):
                eleccion_jugador = "cara"
                break
            elif eleccion in ("CRUZ", "X"):
                eleccion_jugador = "cruz"
                break
            else:
                print("Opción no válida. Escribe 'C' para cara o 'X' para cruz.")
        
        moneda = random.choice(["cara", "cruz"])
        print(f"Resultado del lanzamiento: {moneda}")
        turno_jugador_actual = (eleccion_jugador == moneda)
        print("Comienzas tú." if turno_jugador_actual else "Comienza la máquina.")
        input("Pulsa Enter para continuar...")
        contador_turno = 1

    partida_abandonada = False

""" 
Bucle principal del juego.
"""
    while True:
        print(f"--- Turno {contador_turno} ---")
        
        if turno_jugador_actual:
            continuar, _ = turno_jugador(
                tablero_ataque_jugador, barcos_maquina,
                tablero_defensa_maquina, alias_jugador,
                tablero_defensa_jugador
            )
            if continuar == "salir":
                guardar_partida((
                    alias_jugador, tablero_defensa_jugador, barcos_jugador,
                    tablero_ataque_jugador, tablero_defensa_maquina,
                    barcos_maquina, tablero_ataque_maquina,
                    turno_jugador_actual, contador_turno
                ))
                partida_abandonada = True
                break
            
            if todos_barcos_hundidos(barcos_maquina):
                print("¡Enhorabuena! Bien jugado. ¡Has ganado a la máquina!")
                break
            
            if not continuar:
                turno_jugador_actual = False

        else:
            continuar, _ = turno_maquina(
                tablero_ataque_maquina, barcos_jugador,
                tablero_defensa_jugador
            )
            
            if todos_barcos_hundidos(barcos_jugador):
                print("Game Over - La máquina ha ganado al hombre")
                break
            
            if not continuar:
                turno_jugador_actual = True

        contador_turno += 1

"""
Finalizar partida.
"""
    if not partida_abandonada:
        if os.path.exists(ARCHIVO_PARTIDA):
            os.remove(ARCHIVO_PARTIDA)
        print("--- Tableros finales ---")
        imprimir_tableros_lado(tablero_defensa_jugador, tablero_ataque_jugador, alias_jugador, mostrar_barcos=True)
        print("Gracias por jugar.")

""" 
Ejecutar juego
"""

if __name__ == "__main__":
    juego_principal()


Se ha encontrado una partida guardada.
